In [1]:
import os
import csv
import json
import urllib
import requests
import random
from pprint import pprint
from random import sample
from urllib.request import urlretrieve
from k12libs.utils.nb_easy import k12ai_get_top_dir, RACEURL
from k12libs.utils.nb_easy import K12AI_DATASETS_ROOT

## 开发调试

In [2]:
R_PREFIX = 'https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage'
L_PREFIX = '/raceai/data/datasets/rgarbage'
CSV_FILE = f'{R_PREFIX}/test.csv'

API_INFERENCE = f'{RACEURL}/raceai/framework/inference'

In [3]:
csv_file = urlretrieve(CSV_FILE, os.path.join('/tmp/', os.path.basename(CSV_FILE)))
with open(csv_file[0], 'r') as fr:
    reader = csv.DictReader(fr)
    data = [row for row in reader]
data_10  = sample(data, 10) 
data_10[0]['image_path']

'imgs/有害垃圾/有害垃圾_电池/img_电池_446.jpeg'

In [4]:
model = 'Resnet18'
root_dir = '/raceai/data/tmp/pl_rgarbage_resnet18'
input_size = 224
mean = [
    0.6535,
    0.6132,
    0.5643
]
std = [
    0.2165,
    0.2244,
    0.2416
]
num_classes = 4
resume_weights = f"/raceai/data/ckpts/rgarbage/pl_resnet18_acc70.pth"

reqdata = '''{
    "task": "cls.inference.pl",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.PathListDataLoader",
            "params": {
                "data_source": %s,
                "dataset": {
                    "class_name": "raceai.data.PredictListImageDataset",
                     "params": {
                         "input_size": input_size,
                         "mean": mean,
                         "std": std
                     }
                 },
                "sample": {
                    "batch_size": 32,
                    "num_workers": 4,
                }
             }
        },
        "model": {
            "class_name": f"raceai.models.backbone.{model}",  
            "params": {
                "device": 'gpu',
                "num_classes": num_classes,
                "weights": False
            }
        },
        "trainer": {
            "default_root_dir": root_dir,
            "gpus": 1,
            "resume_from_checkpoint": resume_weights
        }
    }
}'''

### Single Image

In [5]:
for row in data_10:
    imgpath = os.path.join(L_PREFIX, row['image_path'])
    cfg = eval(reqdata % ("[\"" + imgpath + "\"]"))
    resdata = json.loads(requests.post(url=API_INFERENCE, json=cfg).text)
    if resdata['errno'] == 0:
        result = resdata['result'][0]
        fname = result['image_path']
        probs = result['probs']
        print('{}: {} vs {}'.format(fname, row['label'], probs.index(max(probs))))

/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_电池/img_电池_446.jpeg: 3 vs 3
/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_药瓶/红花油_有害垃圾/img_红花油_28.jpeg: 3 vs 3
/raceai/data/datasets/rgarbage/imgs/厨余垃圾/厨余垃圾_面包/汉堡_厨余垃圾/img_汉堡_163.jpeg: 1 vs 1
/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_纽扣电池/img_纽扣电池_28.jpeg: 3 vs 3
/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_电池/img_电池_572.jpeg: 3 vs 3
/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_药膏/img_药膏_19.jpeg: 3 vs 3
/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_电池/img_电池_604.jpeg: 3 vs 0
/raceai/data/datasets/rgarbage/imgs/可回收物/可回收物_水杯/不锈钢杯子_可回收物/img_不锈钢杯子_201.jpeg: 2 vs 2
/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_药瓶/碘伏空瓶_有害垃圾/img_碘伏空瓶_174.jpeg: 3 vs 3
/raceai/data/datasets/rgarbage/imgs/可回收物/可回收物_书/img_书_123.jpeg: 2 vs 2


### Batch Images

In [6]:
images = [os.path.join(L_PREFIX, row['image_path']) for row in data_10]
labels = [row['label'] for row in data_10]
cfg = eval(reqdata % json.dumps(images, ensure_ascii=False))
resdata = json.loads(requests.post(url=API_INFERENCE, json=cfg).text)
if resdata['errno'] == 0:
    for item, label in zip(resdata['result'], labels):
        fname = item['image_path']
        probs = item['probs']
        print('{}: {} vs {}'.format(fname, label, probs.index(max(probs))))
resdata

/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_电池/img_电池_446.jpeg: 3 vs 3
/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_药瓶/红花油_有害垃圾/img_红花油_28.jpeg: 3 vs 3
/raceai/data/datasets/rgarbage/imgs/厨余垃圾/厨余垃圾_面包/汉堡_厨余垃圾/img_汉堡_163.jpeg: 1 vs 1
/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_纽扣电池/img_纽扣电池_28.jpeg: 3 vs 3
/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_电池/img_电池_572.jpeg: 3 vs 3
/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_药膏/img_药膏_19.jpeg: 3 vs 3
/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_电池/img_电池_604.jpeg: 3 vs 0
/raceai/data/datasets/rgarbage/imgs/可回收物/可回收物_水杯/不锈钢杯子_可回收物/img_不锈钢杯子_201.jpeg: 2 vs 2
/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_药瓶/碘伏空瓶_有害垃圾/img_碘伏空瓶_174.jpeg: 3 vs 3
/raceai/data/datasets/rgarbage/imgs/可回收物/可回收物_书/img_书_123.jpeg: 2 vs 2


{'errno': 0,
 'result': [{'image_id': '-1',
   'image_path': '/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_电池/img_电池_446.jpeg',
   'probs': [0.09876822680234909,
    0.017168864607810974,
    0.03145726025104523,
    0.8526056408882141]},
  {'image_id': '-1',
   'image_path': '/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_药瓶/红花油_有害垃圾/img_红花油_28.jpeg',
   'probs': [0.005480817519128323,
    0.12422040849924088,
    0.00048303836956620216,
    0.8698158264160156]},
  {'image_id': '-1',
   'image_path': '/raceai/data/datasets/rgarbage/imgs/厨余垃圾/厨余垃圾_面包/汉堡_厨余垃圾/img_汉堡_163.jpeg',
   'probs': [0.30165186524391174,
    0.4432477056980133,
    0.2020927369594574,
    0.05300763621926308]},
  {'image_id': '-1',
   'image_path': '/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾_纽扣电池/img_纽扣电池_28.jpeg',
   'probs': [0.009896723553538322,
    1.7030284652719274e-05,
    0.012001194059848785,
    0.9780850410461426]},
  {'image_id': '-1',
   'image_path': '/raceai/data/datasets/rgarbage/imgs/有害垃圾/有害垃圾

## 前端对接

In [7]:
TEST_JSON_FILE = f'{R_PREFIX}/test.json'
TEST_JSON_FILE

'https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage/test.json'

### 前端获取测试集的所有图片及对应标号

In [8]:
test_json_file = urlretrieve(TEST_JSON_FILE, os.path.join('/tmp/', os.path.basename(TEST_JSON_FILE)))
with open(test_json_file[0], 'r') as fr:
    test_data = json.load(fr)
test_data[0]

{'image_path': 'imgs/有害垃圾/有害垃圾_纽扣电池/img_纽扣电池_19.jpeg', 'label': 3}

### 前端选择图片数量及AI难度级别

In [9]:
IMAGE_COUNTS = [5, 10, 15, 20]
MODEL_LEVELS = [
    'pl_resnet18_acc70.pth',
    'pl_resnet18_acc80.pth',
    'pl_resnet50_acc90.pth'
]

select_image_count = IMAGE_COUNTS[1]
select_model_level = MODEL_LEVELS[1]

### 随机指定数量的图片

In [10]:
select_images = random.choices(test_data, k=10)
select_images[:2]

[{'image_path': 'imgs/其他垃圾/其他垃圾_笔/笔芯_其他垃圾/img_笔芯_145.jpeg', 'label': 0},
 {'image_path': 'imgs/有害垃圾/有害垃圾_电池/img_电池_538.jpeg', 'label': 3}]

### 参数配置

#### 难度级别

当难度为**低**级别时:

    cfg.model.class_name = raceai.models.backbone.Resnet18
    
    cfg.trainer.resume_from_checkpoint = /raceai/data/ckpts/rgarbage/pl_resnet18_acc70.pth
    
当难度为**中**级别时:

    cfg.model.class_name = raceai.models.backbone.Resnet18
    
    cfg.trainer.resume_from_checkpoint = /raceai/data/ckpts/rgarbage/pl_resnet18_acc80.pth

    
当难度为**高**级别时:

    cfg.model.class_name = raceai.models.backbone.Resnet50
    
    cfg.trainer.resume_from_checkpoint = /raceai/data/ckpts/rgarbage/pl_resnet50_acc90.pth

#### 图片编号

格式:

```
    cfg.data.params.data_source = [
        {"image_path": "xxxx", "image_id": "yyyy"},
        {"image_path": "xxxx", "image_id": "yyyy"},
        {"image_path": "xxxx", "image_id": "yyyy"}
        ...
    ]
```

In [11]:
sources = []
for idx, item in enumerate(select_images):
    sources.append({'image_path': f'{R_PREFIX}/{item["image_path"]}', 'image_id': str(idx)}) 
sources

[{'image_path': 'https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage/imgs/其他垃圾/其他垃圾_笔/笔芯_其他垃圾/img_笔芯_145.jpeg',
  'image_id': '0'},
 {'image_path': 'https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage/imgs/有害垃圾/有害垃圾_电池/img_电池_538.jpeg',
  'image_id': '1'},
 {'image_path': 'https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage/imgs/其他垃圾/其他垃圾_厨房手套/img_厨房手套_106.jpeg',
  'image_id': '2'},
 {'image_path': 'https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage/imgs/有害垃圾/有害垃圾_胶水/img_胶水_44.jpeg',
  'image_id': '3'},
 {'image_path': 'https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage/imgs/有害垃圾/有害垃圾_电池/img_电池_72.jpeg',
  'image_id': '4'},
 {'image_path': 'https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage/imgs/有害垃圾/有害垃圾_电池/img_电池_120.jpeg',
  'image_id': '5'},
 {'image_path': 'https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage/imgs/厨余垃圾/厨余垃圾_面包/汉堡_厨余垃圾/img_汉堡_300.jpeg',
  'image_id': '6'},
 {'image_path': 'https://raceai.s3.didiyunapi.com/data/data

#### 样例模板

In [15]:
reqdata = '''{
    "task": "cls.inference.pl",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.PathListDataLoader",
            "params": {
                "data_source": %s,
                "dataset": {
                    "class_name": "raceai.data.PredictListImageDataset",
                     "params": {
                         "input_size": 224,
                         "mean": [0.6535, 0.6132, 0.5643],
                         "std": [0.2165, 0.2244, 0.2416]
                     }
                 },
                "sample": {
                    "batch_size": 32,
                    "num_workers": 4,
                }
             }
        },
        "model": {
            "class_name": "raceai.models.backbone.Resnet18",  
            "params": {
                "device": 'gpu',
                "num_classes": 4,
                "weights": False
            }
        },
        "trainer": {
            "default_root_dir": "/raceai/data/tmp/pl_rgarbage_resnet18",
            "gpus": 1,
            "resume_from_checkpoint": "/raceai/data/ckpts/rgarbage/pl_resnet18_acc80.pth"
        }
    }
}''' % json.dumps(sources, ensure_ascii=False)

print(reqdata)

{
    "task": "cls.inference.pl",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.PathListDataLoader",
            "params": {
                "data_source": [{"image_path": "https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage/imgs/其他垃圾/其他垃圾_笔/笔芯_其他垃圾/img_笔芯_145.jpeg", "image_id": "0"}, {"image_path": "https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage/imgs/有害垃圾/有害垃圾_电池/img_电池_538.jpeg", "image_id": "1"}, {"image_path": "https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage/imgs/其他垃圾/其他垃圾_厨房手套/img_厨房手套_106.jpeg", "image_id": "2"}, {"image_path": "https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage/imgs/有害垃圾/有害垃圾_胶水/img_胶水_44.jpeg", "image_id": "3"}, {"image_path": "https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage/imgs/有害垃圾/有害垃圾_电池/img_电池_72.jpeg", "image_id": "4"}, {"image_path": "https://raceai.s3.didiyunapi.com/data/datasets/cv/rgarbage/imgs/有害垃圾/有害垃圾_电池/img_电池_120.jpeg", "image_id": "5"}, {"image_path": "https://race

In [13]:
API_INFERENCE

'http://116.85.5.40:9119/raceai/framework/inference'

### 接口请求/响应

请求接口:
   
    http://116.85.5.40:9119/raceai/framework/inference

返回内容:

```json
{
   "errno": 0,   // 正确为0, 其他非0
   "result": [
       {
           "image_id": "yyyy",  // 前端做的图片标识
           "image_path": "xxxx", // 调试用, 可忽略
           "probs": [ // 预测每个分类的概率, 可将概率最大的作为最终结果
                0.01, // "其他垃圾",
                0.91, // "厨余垃圾", 概率最大
                0.02, // "可回收物",
                0.07  // "有害垃圾"
           ]
       },
       {
       
       },
       {
       
       },
   ]
}
```

In [14]:
resdata = json.loads(requests.post(url=API_INFERENCE, json=eval(reqdata)).text)
resdata

{'errno': 0,
 'result': [{'image_id': '0',
   'image_path': '/tmp/tmpugq52aj0/img_%E7%AC%94%E8%8A%AF_145.jpeg',
   'probs': [0.6848078966140747,
    0.0032496743369847536,
    0.21647831797599792,
    0.09546408802270889]},
  {'image_id': '1',
   'image_path': '/tmp/tmpugq52aj0/img_%E7%94%B5%E6%B1%A0_538.jpeg',
   'probs': [1.560087730467785e-05,
    4.718430864159018e-05,
    0.0004344164044596255,
    0.9995027780532837]},
  {'image_id': '2',
   'image_path': '/tmp/tmpugq52aj0/img_%E5%8E%A8%E6%88%BF%E6%89%8B%E5%A5%97_106.jpeg',
   'probs': [0.13916432857513428,
    0.08412338048219681,
    0.1536247283220291,
    0.6230875849723816]},
  {'image_id': '3',
   'image_path': '/tmp/tmpugq52aj0/img_%E8%83%B6%E6%B0%B4_44.jpeg',
   'probs': [6.363891316141235e-06,
    5.554537274576887e-09,
    2.1784214823128423e-06,
    0.9999914169311523]},
  {'image_id': '4',
   'image_path': '/tmp/tmpugq52aj0/img_%E7%94%B5%E6%B1%A0_72.jpeg',
   'probs': [0.00875693280249834,
    0.04692791774868965,
   